In [5]:
# ライブラリをインポート
from google.cloud import speech
import io
import os
from pydub import AudioSegment

In [39]:
# 文字起こし関数
def transcription_wab(audio_file):
    # 音声ファイルの読み込み
    with io.open(audio_file, 'rb') as f:
        content = f.read()

    # APIパラメータの作成
    audio = speech.RecognitionAudio(content = content)
    config = speech.RecognitionConfig(
        # 都度エンコーディングする場合は、LINEAR16 しない場合は、ENCODING_UNSPECIFIED
        # hertzはwavファイルによっては変更する必要があるかもしれないです。自分の場合は32000に指定しろと言われました
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 48000,
        language_code = "ja-JP")

    # APIの呼び出し
    client = speech.SpeechClient()
    response = client.recognize(config = config, audio = audio)

    # 結果の表示
    for result in response.results:
        print(result.alternatives[0].transcript)


In [17]:
# keyを指定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../Transcription/key/credentials.json'
# 実際に実行したいファイルを指定し、文字起こし関数を実行
transcription_wab('../Transcription/audio/20111118meeting.wav')

InvalidArgument: 400 Request payload size exceeds the limit: 10485760 bytes.

In [30]:
import wave
import struct
from scipy import fromstring, int16
import numpy as np
import math

# wavファイルを59秒間隔で分割する関数
def cut_wav(audio_chang_wav, time, wav_cut_dir):
    wr = wave.open(audio_chang_wav, "r")

    # wav情報を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr
    integer = math.floor(total_time)
    t = int(time)
    frames = int(ch * fr * t)
    # 小数点切り上げ（1分に満たない最後のシーンを出力するため）
    num_cut = int(math.ceil(integer / t))
    data = wr.readframes(wr.getnframes())
    wr.close()

    X = np.frombuffer(data, dtype=int16)

    for i in range(num_cut):
        outf = wav_cut_dir + str(i) + ".wav"
        start_cut = int(i * frames)
        end_cut = int(i * frames + frames)
        print(start_cut)
        print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, "w")
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()
        print('終了しました')

In [40]:
# もしかすると、時間の長い音声ファイルは分割しなければ文字起こしできない可能性があるのでテスト
sound = AudioSegment.from_file('../Transcription/audio/20111118meeting.wav', format='wav')
print(len(sound))
# msで記述されている（5~1000秒の切り出し）
sound1 = sound[5000:1000000]
sound1.export('output.wav', format='wav')

0
1920000
終了しました
1920000
3840000
終了しました
3840000
5760000
終了しました
5760000
7680000
終了しました
7680000
9600000
終了しました
9600000
11520000
終了しました
11520000
13440000
終了しました
13440000
15360000
終了しました
15360000
17280000
終了しました
17280000
19200000
終了しました
19200000
21120000
終了しました
21120000
23040000
終了しました
23040000
24960000
終了しました
24960000
26880000
終了しました
26880000
28800000
終了しました
28800000
30720000
終了しました
30720000
32640000
終了しました


In [ ]:
# APIに直接遅れるのが60秒未満なので、59秒で設定している
# 1分以上の場合は、uriでの扱いになる？
time = 59
audio_change_wav = 'output.wav'
wav_cut_dir = '../Transcription/test_file/'
cut_wav(audio_change_wav,time,  wav_cut_dir)

In [ ]:
# mp3 fileをwavに変換したほうが良さそうなので
# ffmpeg を入れないとエラーになりそう
mp3_audio = AudioSegment.from_mp3('../Transcription/audio/001-sibutomo.mp3')
mp3_audio.export('test-audio.wav', format='wav')
